In [18]:
import re 
import tweepy 
from tweepy import OAuthHandler 
from textblob import TextBlob 

In [82]:
# keys and tokens from the Twitter Dev Console
consumer_key = 'x5UHcmFBioYxRCSTEdinjP9aE'
consumer_secret = 'dBdbpYwADt5TeMn6vSQZ6RAY2grsMGD5H01y5KBhlC1p1MJQJm'
access_token = '224341098-cQx598b8K7cgfyXcsWbjT7dTXSJkOobd23z3oB4t'
access_token_secret = 'vKA6xxoRqhzFhdjnYcsBtLWwv9gSaEy5w2niIIBnbmbr5'

In [88]:
class TwitterClient(object):
    '''
    Twitter API client to access tweets for sentiment analysis
    '''

    def __init__(self):
        '''
        Twitter API client constructor 
        '''
        # keys and tokens from the Twitter Dev Console
        consumer_key = 'YMmcZY8h2gyKXR64MPO1gR2b3'
        consumer_secret = 'uHBh1E4lPuf2p5wjM7Hvvg7WERIBNi5cSdFrdKWWKYDr9aChsS'
        access_token = '224341098-sFMWAD1q9d9Kf4xtL71RtNvQFcfvDlrVW77KUkZe'
        access_token_secret = 'RAeBe9rfDViX3F04rwPsdBp3NTNrZlfytTpVSAEJbFPa3'
        # attempt authentication 
        try:
			# create OAuthHandler object 
            self.auth = OAuthHandler(consumer_key, consumer_secret) 
			# set access token and secret 
            self.auth.set_access_token(access_token, access_token_secret) 
			# create tweepy API object to fetch tweets 
            self.api = tweepy.API(self.auth) 
        except: 
            print("Error: Authentication Failed")
    
    def clean_tweet(self, tweet): 
	    ''' 
		Utility function to clean tweet text by removing links, special characters 
		using simple regex statements. 
		'''
	    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t]) |(\w+:\/\/\S+)", " ", tweet).split())

    
    def get_tweet_sentiment(self, tweet): 

        ''' 
        Utility function to classify sentiment of passed tweet 
        using textblob's sentiment method 
        '''
        
        #create TextBlob object of passed tweet text
        
        analysis = TextBlob(self.clean_tweet(tweet))
        
        # set sentiment 
        if analysis.sentiment.polarity > 0: 
            return 'positive'
        elif analysis.sentiment.polarity == 0: 
            return 'neutral'
        else: 
            return 'negative'
            

    def get_tweets(self, query, count = 10): 
        ''' 
		Main function to fetch tweets and parse them. 
        '''
		# empty list to store parsed tweets
        tweets = []
        try: 
            # call twitter api to fetch tweets 
            fetched_tweets = self.api.search(q = query, count = count) 	# parsing tweets one by one 
            for tweet in fetched_tweets: 
                # empty dictionary to store required params of a tweet 
                parsed_tweet = {} 		# saving text of tweet 
                parsed_tweet['text'] = tweet.text 
                # saving sentiment of tweet 
                parsed_tweet['sentiment'] = self.get_tweet_sentiment(tweet.text) 		# appending parsed tweet to tweets list 
                if tweet.retweet_count > 0: 
                    # if tweet has retweets, ensure that it is appended only once 
                    if parsed_tweet not in tweets: 
                        tweets.append(parsed_tweet) 
                else: 
                    tweets.append(parsed_tweet) 	
                # return parsed tweets 
            return tweets 
        except tweepy.TweepError as e: 
            # print error (if any) 
            print("Error : " + str(e)) 


In [160]:
def main(): 
	# creating object of TwitterClient Class 
	api = TwitterClient() 
	# calling function to get tweets 
	tweets = api.get_tweets(query = 'Ayodhya Verdict', count = 10000)	 
    # picking neutral tweets from tweets
	neutral_tweets = [tweet for tweet in tweets if tweet['sentiment'] == 'neutral'] 
    # picking positive tweets from tweets
	postive_tweets = [tweet for tweet in tweets if tweet['sentiment'] == 'positive'] 
    # picking negative tweets from tweets
	negative_tweets = [tweet for tweet in tweets if tweet['sentiment'] == 'negative'] 
	# percentage of neutral tweets 
	print("Neutral tweets percentage: {} % \ ".format(100*len(neutral_tweets)/len(tweets)))
	# percentage of positive tweets 
	print("Positive tweets percentage: {} %".format(100*len(postive_tweets)/len(tweets)))
	# percentage of negative tweets 
	print("Negative tweets percentage: {} %".format(100*len(negative_tweets)/len(tweets))) 
	# printing first 5 positive tweets 
	print("\n\nPositive tweets:") 
	for tweet in postive_tweets[:10]: 
		print(tweet['text']) 	
	# printing first 5 negative tweets 
	print("\n\nNegative tweets:") 
	for tweet in negative_tweets[:10]: 
		print(tweet['text']) 

In [161]:
if __name__ == "__main__": 
    # calling main function 
    main()

Neutral tweets percentage: 63.768115942028984 % \ 
Positive tweets percentage: 31.884057971014492 %
Negative tweets percentage: 4.3478260869565215 %


Positive tweets:
RT @narendramodi: The Honourable Supreme Court has given its verdict on the Ayodhya issue. This verdict shouldn’t be seen as a win or loss…
RT @sgurumurthy: Ayodhya unanimous verdict: the SC judgement is strictly legal and constitutional without getting into religious issues or…
RT @OpIndia_com: This astrologer had predicted exactly when Ram Mandir in Ayodhya will be a reality
https://t.co/FLfJg02Vca
RT @Sanjay_Dixit: Parasaran, the 92-year-old lawyer who stood for hours in courtroom arguing for Ram Lalla is probably the most well versed…
RT @thepeeinghuman: Love will always get noticed more than hate. Big thank you to everyone who participated in #hindumuslimbhaibhai https:/…
RT @sushantsareen: What is more satisfying than the #Ayodhya verdict is the intense heartburn and apoplectic reaction of The Wire and other…
RT @m